# BWT for bioinformatics
Let's apply the BWT to finding strings in a FASTA file. We will combine all precalculations at once so we can process the FASTA in a single pass.

There are a few other minor differences from the methods described in _[Implementing BWT](Implementing BWT.ipynb)_:

 * We assume the alphabet consists of $\Sigma = [A, C, G, T]$ instead of building it from $X$.
 * Bitcounts will start at 0 instead of 1
 * $C(\alpha) =$ first BWT entry starting with $\alpha$ (1 greater than the old $C(\alpha)$)
 * The calculation of $\underline{R}$ now simplifies to $\underline{R} = O(a,i)$


In [1]:
# For all strings in FASTA:
FASTA = {
    "1" : "ATTGCTC$",
    "2" : "AGAGCTCACTG$",
    "N" : "TGACTCACTG$"
}

# This could be derived from all FASTA strings, but we already know the answer
bases = 'ACGT'

# Keep a flyweight in the century table for every (mod) entries
mod = 3

In [12]:
# Build the suffix array, B, the C(a) table, supercontig:position flyweights, n, 
# and century bits simultaneously

from copy import deepcopy

suffixes = []
n = 0
for sc in FASTA:
    # n = sum of all supercontig lengths
    n += len(FASTA[sc])
    # suffix entry == [suffix, supercontig, position, century bit, B]
    suffixes.extend([
        [FASTA[sc][i:], sc, i, 0 if i%mod else 1, FASTA[sc][i-1]] for i in range(len(FASTA[sc]))
    ])

# In the real world, this will be a parallel sort
suffixes = sorted(suffixes)

# Now that we have sorted the suffix table, make the bitcounts and century counts
centcount = [0]

# Calculate bitcounts, saving start entry of each base
bitcounts = [{'A':0, 'C':0, 'G':0, 'T':0}]

# Keep the B string
B = ''
Ca = {}
bases2go = bases
century = []

print "# sc:p  c C   A C G T  B suffix"
for i, (suffix, sc, pos, c, b) in enumerate(suffixes):
    # Build the B string
    B += b
    
    # Accumulate centcounts and century table entries
    centcount.append(centcount[-1] + c)
    if(c):
        century.append([sc, pos])
    
    # # C(a) = first position of a in the BWT
    if bases2go and not bases2go[0] in Ca and suffix[0][0] == bases2go[0]:
        Ca[bases2go[0]] = i
        bases2go = bases2go[1:]
        
    # bitcounts
    prev = deepcopy(bitcounts[i])
    this = {}
    
    for base in bases:
        this[base] = prev[base]
        
    print "{: <3}{}:{: <3}{} {: <3} {} {} {} {}  {} {}".format(
        i, sc, pos, c, centcount[-2], 
        this['A'], this['C'], this['G'], this['T'], 
        b, suffix
    )

    if(b != '$'):
        this[b] = this[b] + 1
        
    bitcounts.append(this)

# Done. Now discard the suffixes!
suffixes[:] = []

print
print 'Ca: {}'.format(Ca)
print 'B == {}'.format(B)
print 'n == {}'.format(n)
print 'suffixes == {}'.format(suffixes)
print '\nCentury:'
print '#  sc:pos'
for i, (sc, pos) in enumerate(century):
    print '{: <3}{}:{}'.format(i, sc, pos)

# sc:p  c C   A C G T  B suffix
0  1:7  0 0   0 0 0 0  C $
1  2:11 0 0   0 1 0 0  G $
2  N:10 0 0   0 1 1 0  G $
3  N:2  0 0   0 1 2 0  G ACTCACTG$
4  2:7  0 0   0 1 3 0  C ACTG$
5  N:6  1 0   0 2 3 0  C ACTG$
6  2:0  1 1   0 3 3 0  $ AGAGCTCACTG$
7  2:2  0 2   0 3 3 0  G AGCTCACTG$
8  1:0  1 2   0 3 4 0  $ ATTGCTC$
9  1:6  1 3   0 3 4 0  T C$
10 2:6  1 4   0 3 4 1  T CACTG$
11 N:5  0 5   0 3 4 2  T CACTG$
12 1:4  0 5   0 3 4 3  G CTC$
13 2:4  0 5   0 3 5 3  G CTCACTG$
14 N:3  1 5   0 3 6 3  A CTCACTG$
15 2:8  0 6   1 3 6 3  A CTG$
16 N:7  0 6   2 3 6 3  A CTG$
17 2:10 0 6   3 3 6 3  T G$
18 N:9  1 6   3 3 6 4  T G$
19 N:1  0 7   3 3 6 5  T GACTCACTG$
20 2:1  0 7   3 3 6 6  A GAGCTCACTG$
21 1:3  1 7   4 3 6 6  T GCTC$
22 2:3  1 8   4 3 6 7  A GCTCACTG$
23 1:5  0 9   5 3 6 7  C TC$
24 2:5  0 9   5 4 6 7  C TCACTG$
25 N:4  0 9   5 5 6 7  C TCACTG$
26 2:9  1 9   5 6 6 7  C TG$
27 N:8  0 10  5 7 6 7  C TG$
28 N:0  1 10  5 8 6 7  $ TGACTCACTG$
29 1:2  0 11  5 8 6 7  T TGCTC$
30 1:1  0 11  5

In [3]:
# O(a,i): lookup r(W) in the bitcounts table
def O(a, i):
    return bitcounts[i][a]

# SA value: compute [i,j] for W
def SA(w):
    # Start with the whole range
    r = 0
    R = n

    for i in range(len(w)-1, -1, -1):
        a = w[i]
        r = Ca[a] + O(a, r) 
        R = Ca[a] + O(a, R) 

        # Exit early if no match
        if r >= R:
            return [r, R]
        
    return [r, R]

In [4]:
# Helper to print SA[i,j] in [inclusive, exclusive) format
def xSA(W):
    return "'{}': {})".format(W, str(SA(W))[:-1])
    
for base in bases:
    print xSA(base)
print 

queries = [
    'GAG',   # j - i == 1, exactly one match
    'CTGA',  # i >= j, not in X
    'TCAC',  # i < j, more than one match
    '',      # empty string, full range
]

for q in queries:
    print xSA(q)

'A': [3, 9)
'C': [9, 17)
'G': [17, 23)
'T': [23, 31)

'GAG': [20, 21)
'CTGA': [17, 17)
'TCAC': [24, 26)
'': [0, 31)


In [5]:
# For a given substring, return [supercontig, position]
def w_to_fasta(W):
    (i, j) = SA(W)
    
    if j > i:
        print 'SA({}) == [{},{})'.format(W,i,j)
    else:
        print 'SA({}) == not found'.format(W)
        return []
    
    reply = []
    
    for k in range(i, j):
        e = k  # e is the bwt entry under examination 
        w = W  # we will be pushing bases to the front of w
        d = 0  # distance from century entry (# of pushes)

        # No need to store the full century bit table: if centcount goes up on the next entry,
        # then this is a century entry.
        while centcount[e + 1] - centcount[e] == 0:
            # a == which letter to push front
            a = B[e]
            # e == next entry to jump to
            e = Ca[a] + O(a, e)
            # d == distance from original position
            d += 1
        (sc, pos) = century[centcount[e]]
        reply.append([sc, pos + d])
        
    return sorted(reply)

def find_in_fasta(W):
    found = ''
    for (sc, begin) in w_to_fasta(W):
        end = begin + len(W)
        found = FASTA[sc][begin:end]
        if W != found:
            raise RuntimeError('{}[{}:{}) != {}'.format(sc, begin, end, found))
        print '  {}[{}:{}) == {}'.format(sc, begin, end, found)
    if found:
        return True
    return False

In [6]:
find_in_fasta('T')
find_in_fasta('CTGA')
find_in_fasta('TCAC')

SA(T) == [23,31)
  1[1:2) == T
  1[2:3) == T
  1[5:6) == T
  2[5:6) == T
  2[9:10) == T
  N[0:1) == T
  N[4:5) == T
  N[8:9) == T
SA(CTGA) == not found
SA(TCAC) == [24,26)
  2[5:9) == TCAC
  N[4:8) == TCAC


True

# Reverse complement lookups
Finally, we will need to consider reverse complements of each query. Simply compute the reverse complement of the query and look it up in the BWT as usual.

In [7]:
def revcomp(seq):
    flip = {
        'A': 'T',
        'C': 'G',
        'G': 'C',
        'T': 'A',
        'N': 'N',
        'a': 't',
        'c': 'g',
        'g': 'c',
        't': 'a',
        'n': 'n'
    }

    reply = ''
    for c in seq:
        reply += flip[c]

    return reply[::-1]

def find_rev(W):
    found = ''
    for (sc, begin) in w_to_fasta(revcomp(W)):
        end = begin + len(W)
        found = FASTA[sc][begin:end]
        if revcomp(W) != found:
            raise RuntimeError('{}[{}:{}) != {}'.format(sc, begin, end, found))
        print '  {}[{}:{}) == {}'.format(sc, begin, end, found)
    if found:
        return True
    return False

In [8]:
print "Forward:"
find_in_fasta('T')

print "\nReverse:"
find_in_fasta(revcomp('T'))

Forward:
SA(T) == [23,31)
  1[1:2) == T
  1[2:3) == T
  1[5:6) == T
  2[5:6) == T
  2[9:10) == T
  N[0:1) == T
  N[4:5) == T
  N[8:9) == T

Reverse:
SA(A) == [3,9)
  1[0:1) == A
  2[0:1) == A
  2[2:3) == A
  2[7:8) == A
  N[2:3) == A
  N[6:7) == A


True

In [9]:
from random import randrange, choice

runs = 200
cache = []
count = 0
print '\n{} random lookups:\n'.format(runs)
for i in range(runs):
    q = ''
    for j in range(randrange(1, 5)):
        q = q + choice(bases)
    while q in cache:
        q = q + choice(bases)
        if len(q) > n:
            break
    cache.append(q)
    if find_in_fasta(q):
        count += 1
        
print '\nFound {} out of {} lookups.'.format(count, runs)


200 random lookups:

SA(CT) == [12,17)
  1[4:6) == CT
  2[4:6) == CT
  2[8:10) == CT
  N[3:5) == CT
  N[7:9) == CT
SA(A) == [3,9)
  1[0:1) == A
  2[0:1) == A
  2[2:3) == A
  2[7:8) == A
  N[2:3) == A
  N[6:7) == A
SA(TTT) == not found
SA(AC) == [3,6)
  2[7:9) == AC
  N[2:4) == AC
  N[6:8) == AC
SA(TA) == not found
SA(TGTC) == not found
SA(AAC) == not found
SA(TCC) == not found
SA(CGG) == not found
SA(GC) == [21,23)
  1[3:5) == GC
  2[3:5) == GC
SA(AT) == [8,9)
  1[0:2) == AT
SA(AG) == [6,8)
  2[0:2) == AG
  2[2:4) == AG
SA(ACG) == not found
SA(ATC) == not found
SA(C) == [9,17)
  1[4:5) == C
  1[6:7) == C
  2[4:5) == C
  2[6:7) == C
  2[8:9) == C
  N[3:4) == C
  N[5:6) == C
  N[7:8) == C
SA(CAG) == not found
SA(CTC) == [12,15)
  1[4:7) == CTC
  2[4:7) == CTC
  N[3:6) == CTC
SA(GCT) == [21,23)
  1[3:6) == GCT
  2[3:6) == GCT
SA(CTG) == [15,17)
  2[8:11) == CTG
  N[7:10) == CTG
SA(CGCT) == not found
SA(ATCT) == not found
SA(ACGT) == not found
SA(CC) == not found
SA(TTC) == not found
SA(C